In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost
!pip install optuna
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.4 MB/s 
     |████████████████████████████████| 81 kB 9.1 MB/s 
     |████████████████████████████████| 209 kB 56.9 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 112 kB 55.2 MB/s 
     |████████████████████████████████| 147 kB 53.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=6e657c6dd9118d8ae9efa6959ec05bd67ac3301069320f4e524bf16d97abc2a6
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://

In [3]:
import numpy as np
import pandas as pd

import sklearn
import sklearn.metrics
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc, precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve

from scipy import stats as st
from random import randrange
from matplotlib import pyplot

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier

import optuna

import shap

In [4]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/ACDF/acdf_final.csv", index_col = 0)
data.head()

,AGE,HEIGHT,WEIGHT,PRSODM,PRBUN,PRCREAT,PRWBC,PRHCT,PRPLATE,TOTHLOS,...,READMISSION1_Yes,RACE_Black or African American,RACE_Hispanic,RACE_Other,RACE_Unknown,RACE_White,LEVELS_Single,COMP_Yes,LOS_Yes,DISCHARGE_Yes
5090996,0.712329,0.743590,0.318681,0.500000,0.085714,0.073253,0.201207,0.703340,0.270111,0.032258,...,0,0,0,0,0,1,1,0,1,0
5089868,0.575342,0.641026,0.263736,0.629032,0.095238,0.046371,0.158954,0.742633,0.194534,0.032258,...,0,0,0,0,0,1,0,0,1,0
5092013,0.397260,0.589744,0.230769,0.580645,0.060714,0.029570,0.191147,0.626719,0.300135,0.043011,...,0,0,0,0,0,1,0,0,1,1
5121528,0.506849,0.512821,0.178022,0.580645,0.086905,0.047715,0.092555,0.679764,0.188322,0.043011,...,0,0,0,0,0,1,0,0,1,0
5157828,0.616438,0.564103,0.252747,0.629032,0.089286,0.027554,0.094567,0.534381,0.202816,0.010753,...,0,0,1,0,0,0,1,0,0,0


In [5]:
#See potential variables.

print(list(data.columns))

['AGE', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRWBC', 'PRHCT', 'PRPLATE', 'TOTHLOS', 'BMI', 'SEX_male', 'SEX_non-binary', 'INOUT_Outpatient', 'TRANST_Transferred', 'TRANST_Unknown', 'SURGSPEC_Orthopedics', 'DIABETES_Yes', 'SMOKE_Yes', 'DYSPNEA_Yes', 'FNSTATUS2_Partially Dependent', 'FNSTATUS2_Totally Dependent', 'FNSTATUS2_Unknown', 'VENTILAT_Yes', 'HXCOPD_Yes', 'ASCITES_Yes', 'HXCHF_Yes', 'HYPERMED_Yes', 'RENAFAIL_Yes', 'DIALYSIS_Yes', 'DISCANCR_Yes', 'WNDINF_Yes', 'STEROID_Yes', 'WTLOSS_Yes', 'BLEEDDIS_Yes', 'TRANSFUS_Yes', 'ASACLAS_2-Mild Disturb', 'ASACLAS_3-Severe Disturb', 'READMISSION1_Yes', 'RACE_Black or African American', 'RACE_Hispanic', 'RACE_Other', 'RACE_Unknown', 'RACE_White', 'LEVELS_Single', 'COMP_Yes', 'LOS_Yes', 'DISCHARGE_Yes']


In [6]:
#Define predictor variables and outcome of interest.

variables = ['AGE', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRWBC', 'PRHCT', 'PRPLATE', 'BMI', 'SEX_male', 'SEX_non-binary', 'INOUT_Outpatient', 'TRANST_Transferred', 'TRANST_Unknown', 'SURGSPEC_Orthopedics', 'DIABETES_Yes', 'SMOKE_Yes', 'DYSPNEA_Yes', 'FNSTATUS2_Partially Dependent', 'FNSTATUS2_Totally Dependent', 'FNSTATUS2_Unknown', 'VENTILAT_Yes', 'HXCOPD_Yes', 'ASCITES_Yes', 'HXCHF_Yes', 'HYPERMED_Yes', 'RENAFAIL_Yes', 'DIALYSIS_Yes', 'DISCANCR_Yes', 'WNDINF_Yes', 'STEROID_Yes', 'WTLOSS_Yes', 'BLEEDDIS_Yes', 'TRANSFUS_Yes', 'ASACLAS_2-Mild Disturb', 'ASACLAS_3-Severe Disturb', 'RACE_Black or African American', 'RACE_Hispanic', 'RACE_Other', 'RACE_Unknown', 'RACE_White', 'LEVELS_Single', 'LOS_Yes']

In [7]:
#Redefine data.

data = data[variables]

In [8]:
#Define predictor variables (x) and outcome of interest (y).

x = data.drop(['LOS_Yes'], axis = 1)
y = data['LOS_Yes']

In [9]:
#Check data shapes.

print(y.shape)
print(x.shape)

(44903,)
(44903, 43)


In [10]:
#Split data into initial train set and test set in 80:20 ratio.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  
#Describe initial train set and test set.

print("Number patients x_train dataset: ", x_train.shape)
print("Number patients y_train dataset: ", y_train.shape)
print("Number patients x_test dataset: ", x_test.shape)
print("Number patients y_test dataset: ", y_test.shape)

Number patients x_train dataset:  (35922, 43)
Number patients y_train dataset:  (35922,)
Number patients x_test dataset:  (8981, 43)
Number patients y_test dataset:  (8981,)


In [11]:
#Split initial train set into final train set and validation set in 75:25 ratio.

train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, test_size = 0.25, random_state = 0)
  
#Describe train and validation sets.

print("Number patients train_x dataset: ", train_x.shape)
print("Number patients train_y dataset: ", train_y.shape)
print("Number patients valid_x dataset: ", valid_x.shape)
print("Number patients valid_y dataset: ", valid_y.shape)

Number patients train_x dataset:  (26941, 43)
Number patients train_y dataset:  (26941,)
Number patients valid_x dataset:  (8981, 43)
Number patients valid_y dataset:  (8981,)


In [12]:
#Describe outcome of interest before resampling.

print("Before resampling, counts of label '1': {}".format(sum(train_y == 1)))
print("Before resampling, counts of label '0': {} \n".format(sum(train_y == 0)))

Before resampling, counts of label '1': 3010
Before resampling, counts of label '0': 23931 



In [13]:
#Apply SMOTE.

from imblearn.over_sampling import SMOTE

resampler = SMOTE()
train_x, train_y = resampler.fit_resample(train_x, train_y)

In [14]:
#Describe outcome of interest after resampling.

print("After resampling, counts of label '1': {}".format(sum(train_y == 1)))
print("After resampling, counts of label '0': {} \n".format(sum(train_y == 0)))

After resampling, counts of label '1': 23931
After resampling, counts of label '0': 23931 



# XGBoost

In [15]:
def objective(trial):

    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective":  trial.suggest_categorical("objective", ["binary:logistic"]),
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        
    xgb_params = {}
    
    for key, value in trial.params.items():
        xgb_params[key] = value

[I 2022-11-21 01:20:01,234] A new study created in memory with name: no-name-268e1226-33e9-4ec5-a8a6-16377f8871d6


[0]	validation-auc:0.724456
[1]	validation-auc:0.724456
[2]	validation-auc:0.724411
[3]	validation-auc:0.724411
[4]	validation-auc:0.724411
[5]	validation-auc:0.724411
[6]	validation-auc:0.724411
[7]	validation-auc:0.724411
[8]	validation-auc:0.724411
[9]	validation-auc:0.724411


[I 2022-11-21 01:20:03,703] Trial 0 finished with value: 0.6676791273098028 and parameters: {'objective': 'binary:logistic', 'booster': 'gbtree', 'lambda': 0.0002623532591987795, 'alpha': 0.07914480155020225, 'max_depth': 6, 'eta': 0.00030148766295521364, 'gamma': 3.907842215039409e-06, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6676791273098028.


[0]	validation-auc:0.748372
[1]	validation-auc:0.740806
[2]	validation-auc:0.733132
[3]	validation-auc:0.739011
[4]	validation-auc:0.731353
[5]	validation-auc:0.733257
[6]	validation-auc:0.73665
[7]	validation-auc:0.727524
[8]	validation-auc:0.723571
[9]	validation-auc:0.723329


[I 2022-11-21 01:20:07,050] Trial 1 finished with value: 0.5942953184888287 and parameters: {'objective': 'binary:logistic', 'booster': 'dart', 'lambda': 0.015776869351741586, 'alpha': 3.5176712121942825e-05, 'max_depth': 4, 'eta': 0.5731734383796065, 'gamma': 0.0010530303721179338, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.2189432280570128e-08, 'skip_drop': 0.0032365042432882167}. Best is trial 0 with value: 0.6676791273098028.


...

[0]	validation-auc:0.763812
[1]	validation-auc:0.767552
[2]	validation-auc:0.768592
[3]	validation-auc:0.768712
[4]	validation-auc:0.768934


[I 2022-11-21 01:21:16,819] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.768399
[1]	validation-auc:0.769213
[2]	validation-auc:0.769386
[3]	validation-auc:0.769642
[4]	validation-auc:0.769361
[5]	validation-auc:0.769183
[6]	validation-auc:0.76911
[7]	validation-auc:0.769117
[8]	validation-auc:0.76917


[I 2022-11-21 01:21:17,316] Trial 99 pruned. Trial was pruned at iteration 9.


Number of finished trials: 100
Best trial:
  Value: 0.719083700223123
  Params: 
    objective: binary:logistic
    booster: gblinear
    lambda: 0.010670271263133628
    alpha: 1.6193422152528249e-07


In [16]:
#Fit XGBoost.

from xgboost import XGBClassifier

xgb = XGBClassifier(**xgb_params)

xgb.fit(train_x, train_y)

XGBClassifier(alpha=1.6193422152528249e-07, booster='gblinear',
              lambda=0.010670271263133628)

In [17]:
#Make predictions on the test set based on the trained XGBoost model.

preds_xgb = xgb.predict(x_test)

probs_xgb = xgb.predict_proba(x_test)
probs_xgb = probs_xgb[:, 1]

In [18]:
#Evaluate XGBoost model.

xgb_precision = precision_score(preds_xgb,y_test)
xgb_recall = recall_score(preds_xgb,y_test)
xgb_f1 = f1_score(preds_xgb,y_test)
xgb_acc = accuracy_score(preds_xgb,y_test)   
xgb_mcc = matthews_corrcoef(y_test, preds_xgb)
xgb_auroc = roc_auc_score(y_test, probs_xgb)

print("Precision: %.3f" % (xgb_precision))
print("Recall: %.3f" % (xgb_recall))
print("F1 Score: %.3f" %(xgb_f1))
print('Accuracy: %.3f' % (xgb_acc))
print('MCC: %.3f' % (xgb_mcc))
print('AUROC: %.3f' % (xgb_auroc))

Precision: 0.672
Recall: 0.213
F1 Score: 0.323
Accuracy: 0.694
MCC: 0.242
AUROC: 0.756


In [19]:
#Evaluate XGBoost model (PRC and AUPRC).

xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, probs_xgb)
xgb_auprc = auc(xgb_recall, xgb_precision)

print('AUPRC: %.3f' % (xgb_auprc))

AUPRC: 0.257


In [20]:
#Recalculate precision and recall for calculation purposes.

xgb_precision = precision_score(preds_xgb,y_test)
xgb_recall = recall_score(preds_xgb,y_test)

xgb_results = [xgb_precision, xgb_recall, xgb_f1, xgb_acc, xgb_mcc, xgb_auroc, xgb_auprc]

In [21]:
#Recalculate precision recall curve for plotting purposes.

xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, probs_xgb)

# LightGBM

In [22]:
#Hyperparameter tuning for LightGBM.

def objective(trial):
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective":  trial.suggest_categorical("objective", ["binary"]),
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type":  trial.suggest_categorical("boosting_type", ["gbdt"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)
    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        
    lgb_params = {}
    
    for key, value in trial.params.items():
        lgb_params[key] = value

[I 2022-11-21 01:21:20,336] A new study created in memory with name: no-name-c814529b-47c5-4507-9747-0800cca0a680
[I 2022-11-21 01:21:23,487] Trial 0 finished with value: 0.5430843341557667 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'lambda_l1': 0.004071459103501498, 'lambda_l2': 0.0004727702953210512, 'num_leaves': 95, 'feature_fraction': 0.7412018375878764, 'bagging_fraction': 0.8776030189908115, 'bagging_freq': 6, 'min_child_samples': 75}. Best is trial 0 with value: 0.5430843341557667.
[I 2022-11-21 01:21:26,040] Trial 1 finished with value: 0.5364264514755341 and parameters: {'objective': 'binary', 'boosting_type': 'gbdt', 'lambda_l1': 0.0015133838972999627, 'lambda_l2': 0.37037887329047386, 'num_leaves': 78, 'feature_fraction': 0.8342954134394472, 'bagging_fraction': 0.7126752089317332, 'bagging_freq': 2, 'min_child_samples': 43}. Best is trial 0 with value: 0.5430843341557667.
[I 2022-11-21 01:21:28,021] Trial 2 finished with value: 0.5429763468334021 and p

Number of finished trials: 100
Best trial:
  Value: 0.7014286633713364
  Params: 
    objective: binary
    boosting_type: gbdt
    lambda_l1: 2.4979101276990208
    lambda_l2: 3.039019408265432e-08
    num_leaves: 2
    feature_fraction: 0.49911918163841645
    bagging_fraction: 0.6263675478864141
    bagging_freq: 7
    min_child_samples: 94


In [23]:
#Fit LightGBM.

import lightgbm as lgb

lgb = lgb.LGBMClassifier(**lgb_params)

lgb.fit(train_x, train_y)

LGBMClassifier(bagging_fraction=0.6263675478864141, bagging_freq=7,
               feature_fraction=0.49911918163841645,
               lambda_l1=2.4979101276990208, lambda_l2=3.039019408265432e-08,
               min_child_samples=94, num_leaves=2, objective='binary')

In [24]:
#Make predictions on the test set based on the trained model.

preds_lgb = lgb.predict(x_test)

probs_lgb = lgb.predict_proba(x_test)
probs_lgb = probs_lgb[:, 1]

In [25]:
#Evaluate LightGBM model.

lgb_precision = precision_score(preds_lgb,y_test)
lgb_recall = recall_score(preds_lgb,y_test)
lgb_f1 = f1_score(preds_lgb,y_test)
lgb_acc = accuracy_score(preds_lgb,y_test)   
lgb_mcc = matthews_corrcoef(y_test, preds_lgb)
lgb_auroc = roc_auc_score(y_test, probs_lgb)
                          
print("Precision: %.3f" % (lgb_precision))
print("Recall: %.3f" % (lgb_recall))
print("F1 Score: %.3f" %(lgb_f1))
print('Accuracy: %.3f' % (lgb_acc))
print('MCC: %.3f' % (lgb_mcc))
print('AUROC: %.3f' % (lgb_auroc))

Precision: 0.831
Recall: 0.193
F1 Score: 0.313
Accuracy: 0.605
MCC: 0.254
AUROC: 0.764


In [26]:
#Evaluate LightGBM model (PRC and AUPRC).

lgb_precision, lgb_recall, _ = precision_recall_curve(y_test, probs_lgb)
lgb_auprc = auc(lgb_recall, lgb_precision)

print('AUPRC: %.3f' % (lgb_auprc))

AUPRC: 0.252


In [27]:
#Recalculate precision and recall for calculation purposes.

lgb_precision = precision_score(preds_lgb,y_test)
lgb_recall = recall_score(preds_lgb,y_test)

lgb_results = [lgb_precision, lgb_recall, lgb_f1, lgb_acc, lgb_mcc, lgb_auroc, lgb_auprc]

In [28]:
#Recalculate precision recall curve for plotting purposes.

lgb_precision, lgb_recall, _ = precision_recall_curve(y_test, probs_lgb)

# CatBoost

In [29]:
#Hyperparameter tuning for CatBoost.

from optuna.integration import CatBoostPruningCallback

def objective(trial: optuna.Trial) -> float:

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "AUC",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "AUC")
    gbm.fit(
        train_x,
        train_y,
        eval_set=[(valid_x, valid_y)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc

if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    cb_params = {}
    
    for key, value in trial.params.items():
        cb_params[key] = value

[I 2022-11-21 01:24:51,413] A new study created in memory with name: no-name-419ae43a-2d91-4481-8dd8-dad869720be6
CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
[I 2022-11-21 01:25:01,487] Trial 0 finished with value: 0.7093627425167583 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.02966096210335524, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.215985307677363}. Best is trial 0 with value: 0.7093627425167583.
CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
[I 2022-11-21 01:25:05,674] Trial 1 finished with value: 0.7088255405662199 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.04461505488626063, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.992787568810996}. Best is trial 0 with value: 0.7093627425167583.
CatBoostPruningCallback is exper

Number of finished trials: 100
Best trial:
  Value: 0.7161540512289822
  Params: 
    objective: CrossEntropy
    colsample_bylevel: 0.05888279602035283
    depth: 5
    boosting_type: Plain
    bootstrap_type: Bernoulli
    subsample: 0.14630926431770105


In [30]:
#Fit CatBoost.

from catboost import CatBoostClassifier

cb = CatBoostClassifier(**cb_params)

cb.fit(train_x, train_y)

0:	learn: 0.6917292	total: 8.5ms	remaining: 8.49s
1:	learn: 0.6819312	total: 34.1ms	remaining: 17s
2:	learn: 0.6795486	total: 63.2ms	remaining: 21s
3:	learn: 0.6700548	total: 86.5ms	remaining: 21.5s
4:	learn: 0.6681464	total: 110ms	remaining: 21.9s
5:	learn: 0.6671527	total: 132ms	remaining: 21.9s
6:	learn: 0.6660488	total: 156ms	remaining: 22.1s
7:	learn: 0.6641832	total: 179ms	remaining: 22.2s
8:	learn: 0.6639512	total: 188ms	remaining: 20.8s
9:	learn: 0.6633325	total: 203ms	remaining: 20s
10:	learn: 0.6551065	total: 225ms	remaining: 20.2s
11:	learn: 0.6542789	total: 252ms	remaining: 20.8s
12:	learn: 0.6537109	total: 277ms	remaining: 21.1s
13:	learn: 0.6460765	total: 307ms	remaining: 21.6s
14:	learn: 0.6385497	total: 338ms	remaining: 22.2s
15:	learn: 0.6380351	total: 360ms	remaining: 22.1s
16:	learn: 0.6370556	total: 384ms	remaining: 22.2s
17:	learn: 0.6370534	total: 398ms	remaining: 21.7s
18:	learn: 0.6309788	total: 419ms	remaining: 21.6s
19:	learn: 0.6304138	total: 444ms	remaining:

In [31]:
#Make predictions on the test set based on the trained model.

preds_cb = cb.predict(x_test)

probs_cb = cb.predict_proba(x_test)
probs_cb = probs_cb[:, 1]

In [32]:
#Evaluate CatBoost model.

cb_precision = precision_score(preds_cb,y_test)
cb_recall = recall_score(preds_cb,y_test)
cb_f1 = f1_score(preds_cb,y_test)
cb_acc = accuracy_score(preds_cb,y_test)   
cb_mcc = matthews_corrcoef(y_test, preds_cb)
cb_auroc = roc_auc_score(y_test, probs_cb)
                          
print("Precision: %.3f" % (cb_precision))
print("Recall: %.3f" % (cb_recall))
print("F1 Score: %.3f" %(cb_f1))
print('Accuracy: %.3f' % (cb_acc))
print('MCC: %.3f' % (cb_mcc))
print('AUROC: %.3f' % (cb_auroc))

Precision: 0.578
Recall: 0.219
F1 Score: 0.318
Accuracy: 0.731
MCC: 0.226
AUROC: 0.749


In [33]:
#Evaluate XGBoost model (PRC and AUPRC).

cb_precision, cb_recall, _ = precision_recall_curve(y_test, probs_cb)
cb_auprc = auc(cb_recall, cb_precision)

print('AUPRC: %.3f' % (cb_auprc))

AUPRC: 0.242


In [34]:
#Recalculate precision and recall for calculation purposes.

cb_precision = precision_score(preds_cb,y_test)
cb_recall = recall_score(preds_cb,y_test)

cb_results = [cb_precision, cb_recall, cb_f1, cb_acc, cb_mcc, cb_auroc, cb_auprc]

In [35]:
#Recalculate precision recall curve for plotting purposes.

cb_precision, cb_recall, _ = precision_recall_curve(y_test, probs_cb)

# Random Forest

In [36]:
#Hyperparameter tuning for Random Forest.

def objective(trial):
    
    param = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "bootstrap": trial.suggest_categorical("bootstrap", ["auto", "sqrt"]),
        "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt","log2", None]),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4, 1),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10, 1),
    }

    rf = RandomForestClassifier(**param)

    rf.fit(
        train_x,
        train_y,
    )

    preds = rf.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        
    rf_params = {}
    
    for key, value in trial.params.items():
        rf_params[key] = value

[I 2022-11-21 01:26:48,849] A new study created in memory with name: no-name-61227c0d-9897-4515-ba14-d67c2aff035d
[I 2022-11-21 01:27:36,450] Trial 0 finished with value: 0.6376440244343895 and parameters: {'criterion': 'gini', 'bootstrap': 'auto', 'max_features': 'log2', 'max_depth': 35, 'n_estimators': 500, 'min_samples_leaf': 3, 'min_samples_split': 5}. Best is trial 0 with value: 0.6376440244343895.
[I 2022-11-21 01:49:23,202] Trial 1 finished with value: 0.517444531529627 and parameters: {'criterion': 'entropy', 'bootstrap': 'auto', 'max_features': None, 'max_depth': 63, 'n_estimators': 1600, 'min_samples_leaf': 4, 'min_samples_split': 7}. Best is trial 0 with value: 0.6376440244343895.


Number of finished trials: 2
Best trial:
  Value: 0.6376440244343895
  Params: 
    criterion: gini
    bootstrap: auto
    max_features: log2
    max_depth: 35
    n_estimators: 500
    min_samples_leaf: 3
    min_samples_split: 5


In [37]:
#Fit Random Forest.

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(**rf_params)

rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap='auto', max_depth=35, max_features='log2',
                       min_samples_leaf=3, min_samples_split=5,
                       n_estimators=500)

In [38]:
#Make predictions on the test set based on the trained model.

preds_rf = rf.predict(x_test)

probs_rf = rf.predict_proba(x_test)
probs_rf = probs_rf[:, 1]

In [39]:
#Evaluate Random Forest model.

rf_precision = precision_score(preds_rf,y_test)
rf_recall = recall_score(preds_rf,y_test)
rf_f1 = f1_score(preds_rf,y_test)
rf_acc = accuracy_score(preds_rf,y_test)   
rf_mcc = matthews_corrcoef(y_test, preds_rf)
rf_auroc = roc_auc_score(y_test, probs_rf)
                          
print("Precision: %.3f" % (rf_precision))
print("Recall: %.3f" % (rf_recall))
print("F1 Score: %.3f" %(rf_f1))
print('Accuracy: %.3f' % (rf_acc))
print('MCC: %.3f' % (rf_mcc))
print('AUROC: %.3f' % (rf_auroc))

Precision: 0.403
Recall: 0.252
F1 Score: 0.310
Accuracy: 0.806
MCC: 0.212
AUROC: 0.759


In [40]:
#Evaluate Random Forest model (PRC and AUPRC).

rf_precision, rf_recall, _ = precision_recall_curve(y_test, probs_rf)
rf_auprc = auc(rf_recall, rf_precision)

print('AUPRC: %.3f' % (rf_auprc))

AUPRC: 0.234


In [41]:
#Recalculate precision and recall for calculation purposes.

rf_precision = precision_score(preds_rf,y_test)
rf_recall = recall_score(preds_rf,y_test)

rf_results = [rf_precision, rf_recall, rf_f1, rf_acc, rf_mcc, rf_auroc, rf_auprc]

In [42]:
#Recalculate precision recall curve for plotting purposes.

rf_precision, rf_recall, _ = precision_recall_curve(y_test, probs_rf)

# ROC, PR, and Calibration Plots

In [ ]:
f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

xgb_fpr, xgb_tpr, _ = roc_curve(y_test, probs_xgb)
pyplot.plot(xgb_fpr, xgb_tpr, label='XGBoost AUROC: {:.3f}'.format(xgb_auroc), color='red')

lgb_fpr, lgb_tpr, _ = roc_curve(y_test, probs_lgb)
pyplot.plot(lgb_fpr, lgb_tpr, label='LightGBM AUROC: {:.3f}'.format(lgb_auroc), color='darkblue')

cb_fpr, cb_tpr, _ = roc_curve(y_test, probs_cb)
pyplot.plot(cb_fpr, cb_tpr, label='CatBoost AUROC: {:.3f}'.format(cb_auroc), color = 'darkgreen')

rf_fpr, rf_tpr, _ = roc_curve(y_test, probs_rf)
pyplot.plot(rf_fpr, rf_tpr, label='Random Forest AUROC: {:.3f}'.format(rf_auroc), color = 'orange')

pyplot.plot([0, 1], [0, 1], linestyle = '--')

pyplot.title('Receiver Operating Characteristic Curve', loc='center', fontsize = 20, fontweight = 'heavy', pad = 20)
pyplot.xlabel('False Positive Rate', fontsize = 16, labelpad = 10)
pyplot.ylabel('True Positive Rate', fontsize = 16, labelpad = 10)
pyplot.tick_params(axis="y",direction="out")
pyplot.tick_params(axis="x",direction="out")

leg = pyplot.legend(loc = 'lower right', fontsize = 12)

pyplot.savefig('/content/drive/MyDrive/ACDF/los_roc_us.png', dpi=300)
pyplot.show()

In [ ]:
f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

pyplot.plot(xgb_recall, xgb_precision, label='XGBoost AUPRC: {:.3f}'.format(xgb_auprc), color = 'red')
pyplot.plot(lgb_recall, lgb_precision, label='LightGBM AUPRC: {:.3f}'.format(lgb_auprc), color = 'darkblue')
pyplot.plot(cb_recall, cb_precision, label='CatBoost AUPRC: {:.3f}'.format(cb_auprc), color = 'darkgreen')
pyplot.plot(rf_recall, rf_precision, label='Random Forest AUPRC: {:.3f}'.format(rf_auprc), color = 'orange')


pyplot.title('Precision Recall Curve', loc='center', fontsize = 20, fontweight = 'heavy', pad = 20)
pyplot.xlabel('Recall', fontsize = 16, labelpad = 10)
pyplot.ylabel('Precision', fontsize = 16, labelpad = 10)
leg = pyplot.legend(loc = 'lower right', fontsize = 12)

pyplot.savefig('/content/drive/MyDrive/ACDF/los_prc_us.png', dpi=300)
pyplot.show()

In [ ]:
f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

x_cal_xgb, y_cal_xgb = calibration_curve(y_test, probs_xgb, n_bins = 10, normalize = True)
x_cal_lgb, y_cal_lgb = calibration_curve(y_test, probs_lgb, n_bins = 10, normalize = True)
x_cal_cb, y_cal_cb = calibration_curve(y_test, probs_cb, n_bins = 10, normalize = True)
x_cal_rf, y_cal_rf = calibration_curve(y_test, probs_rf, n_bins = 10, normalize = True)


pyplot.plot([0, 1], [0, 1], linestyle = '--', label = 'Ideally Calibrated')

pyplot.plot(y_cal_xgb, x_cal_xgb, label = 'XGBoost', color = 'red')
pyplot.plot(y_cal_lgb, x_cal_lgb, label = 'LightGBM', color = 'darkblue')
pyplot.plot(y_cal_cb, x_cal_cb, label = 'CatBoost', color = 'darkgreen')
pyplot.plot(y_cal_rf, x_cal_xgb, label = 'Random Forest', color = 'orange')

pyplot.title('Calibration', loc='center', fontsize = 20, fontweight = 'heavy', pad = 20)
leg = pyplot.legend(loc = 'lower right', fontsize = 12)
pyplot.xlabel('Average Predicted Probability in each bin', fontsize = 16, labelpad = 10)
pyplot.ylabel('Ratio of positives', fontsize = 16, labelpad = 10)

pyplot.savefig('/content/drive/MyDrive/ACDF/los_cal_us.png', dpi=300)
pyplot.show()

# Results Summary

In [ ]:
results = {'XGBoost':xgb_results, 'LightGBM':lgb_results, 'CatBoost':cb_results, 'Random Forest':rf_results}

results = pd.DataFrame(results, columns = ['XGBoost', 'LightGBM', 'CatBoost', 'Random Forest'])

results

In [ ]:
results = pd.DataFrame({'XGBoost':xgb_results, 'LightGBM':lgb_results, 'CatBoost':cb_results, 'Random Forest':rf_results})

results = results.T

results.columns = ['Precision', 'Recall', 'F1', 'Accuracy', 'MCC', 'AUROC', 'AUPRC']

results.to_csv('/content/drive/MyDrive/ACDF/los_results_us.csv')

results

# SHAP Plots

In [ ]:
# Fits the explainer
xgb_explainer = shap.Explainer(xgb.predict, x_test)
lgb_explainer = shap.Explainer(lgb.predict, x_test)
cb_explainer = shap.Explainer(cb.predict, x_test)
rf_explainer = shap.Explainer(rf.predict, x_test)

# Calculates the SHAP values - It takes some time
xgb_shap_values = xgb_explainer(x_test)
lgb_shap_values = lgb_explainer(x_test)
cb_shap_values = cb_explainer(x_test)
rf_shap_values = rf_explainer(x_test)

In [ ]:
from scipy.special import softmax

def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

In [ ]:
shap.plots.bar(xgb_shap_values, max_display = 10, show=False)
pyplot.savefig('/content/drive/MyDrive/ACDF/los_shap_xgb.png', dpi=300)

In [ ]:
shap.plots.bar(lgb_shap_values, max_display = 10, show=False)
pyplot.savefig('/content/drive/MyDrive/ACDF/los_shap_lgb.png', dpi=300)

In [ ]:
shap.plots.bar(cb_shap_values, max_display = 10, show=False)
pyplot.savefig('/content/drive/MyDrive/ACDF/los_shap_cb.png', dpi=300)

In [ ]:
shap.plots.bar(rf_shap_values, max_display = 10, show=False)
pyplot.savefig('/content/drive/MyDrive/ACDF/los_shap_rf.png', dpi=300)